# Ollamaの基本的な使用例

OpenAI/その他のAPI呼び出しをOllamaで置き換える最もシンプルな方法を示します。
エージェントに進む前に、コアコンセプトを理解するのに最適です。

1) 直接API呼び出し
2) Ollama APIパラメータ
3) 基本的なLangChain統合
4) 会話

In [ ]:
import requests
import time

# Ollamaが実行されているか確認しましょう：

In [ ]:
def check_ollama_running():
    """Ollamaが利用可能かチェックする。"""
    try:
        response = requests.get("http://localhost:11434/api/tags", timeout=5)
        return response.status_code == 200
    except:
        return False

if not check_ollama_running():
    print("❌ Ollamaが実行されていません！")
    print("次のコマンドで起動してください: ollama serve")
    print("モデルがあることを確認してください: ollama pull llama3.1:8b")
else:
    print("✅ Ollamaが実行されています\n")
    print("🚀 Ollamaの基本的な使用例")
    print("=" * 40)

# 直接API

# systemとuserの呼び出し：
✅ role: "system"
目的：モデルの初期動作、トーン、または制約を設定します。
使用例：会話が始まる前に、アシスタントの個性、スコープ、または指示を定義します。
モデルは一度読み込み、以降のすべての応答を形成するために使用します。
ほとんどのアプリケーションでは、ユーザーには表示されません。

✅ role: "user"
目的：ユーザーからの実際の質問や入力を表します。
モデルに応答を促します。
アシスタントメッセージが続くことで、対話のやり取りを継続できます。

In [ ]:
def example_1_direct_api(sys_promt, usr_promt):
    # これはOpenAI API呼び出しを置き換えます
    # response = requests.post("http://localhost:11434/api/chat", json={
    #     "model": "llama3.1:8b",
    #     "messages": [
    #         {"role": "system", "content": "You are a helpful assistant."},
    #         {"role": "user", "content": "What is Python?"}
    #     ]
    # })
    response = requests.post("http://localhost:11434/api/chat", json={
        "model": "llama3.1:8b",
        "messages": [{"role": "system", "content": sys_promt},
                     {"role": "user", "content":  usr_promt}
                     ],
        "stream": False  # ストリーミングでないことを確認
    })
    if response.status_code == 200:
        result = response.json()
        print(f"応答: {result["message"]["content"]}")
    else:
        print(f"エラー: {response.status_code}")

In [ ]:
print("例1: 直接API呼び出し")
print("-" * 30)
sys_promt = "あなたは役立つアシスタントです。"
usr_promt = "Pythonとは何ですか？"
example_1_direct_api(sys_promt,usr_promt)

## Ollama APIパラメータ

Ollamaは広範なカスタマイズオプションを提供します。最も一般的に使用されるパラメータは以下の通りです：

| パラメータ | 型 | デフォルト | 説明 |
|-----------|------|---------|-------------|
| `model` | string | - | **必須。** モデル識別子（例：`"llama3.1:8b"`） |
| `messages` | array | - | `/api/chat`用のチャット会話 |
| `stream` | boolean | true | 生成されるにつれて応答をストリーム |
| `temperature` | float | 0.8 | ランダム性（0.0 = 決定的、2.0 = 非常にランダム） |
| `top_p` | float | 0.9 | 核サンプリング（0.1 = 上位10%のトークンのみ） |
| `num_predict` | int | 128 | 生成する最大トークン数（-1 = 無制限） |
| `repeat_penalty` | float | 1.1 | トークンの繰り返しに対するペナルティ |
| `system` | string | - | 動作をガイドするシステムメッセージ |
| `stop` | array | - | これらの文字列で生成を停止 |

In [ ]:
def example_2_with_parameters(temperature, num_predict, usr_promt):
   
    response = requests.post("http://localhost:11434/api/chat", json={
        "model": "llama3.1:8b",
        "messages": [{"role": "user", "content": usr_promt}],
        "temperature": temperature,  # 創造性のための高い温度
        "num_predict": num_predict,   # 応答の長さを制限
        "stream": False  # ストリーミングでないことを確認
    })
    
    if response.status_code == 200:
        result = response.json()["message"]["content"]
        print(f"創造的な応答: {result}")
    else:
        print(f"エラー: {response.status_code}")

In [ ]:
print("\n例2: パラメータ付き")
print("-" * 30)
usr_promt = "ロボットについての創造的な物語を書いてください。"
temperature = 0.8
num_predict = 100
example_2_with_parameters(temperature, num_predict, usr_promt)

# LangChain統合

| 機能 | ChatOllama | 直接API |
|---------|------------|------------|
| `抽象化レベル` | 高 | 低 |
| `使いやすさ` | より簡単 | 手動フォーマットが必要 |
| `カスタマイズ（ヘッダーなど）` | 限定的 | 完全 |
| `依存関係` | LangChainが必要 | requestsのみ | 
| `最適な用途` | RAGパイプライン、高速プロトタイピング | カスタムツール、低レベル制御 |

In [ ]:
def example_3_langchain_basic(usr_promt):    
    try:
        from langchain_community.chat_models import ChatOllama
        
        # ChatOpenAIをChatOllamaで置き換える
        llm = ChatOllama(model="llama3.1:8b", temperature=0.1)
        
        response = llm.invoke(usr_promt)
        print(f"LangChain応答: {response.content}")
        
    except ImportError:
        print("LangChainがインストールされていません。実行してください: pip install langchain langchain-community")
    except Exception as e:
        print(f"エラー: {e}")

In [ ]:
print("\n例3: LangChain統合")
print("-" * 30)
usr_promt = "機械学習を一文で説明してください。"
example_3_langchain_basic(usr_promt)

# 会話コンテキストの維持

言語モデルとの会話では、異なるロール間でメッセージが交換されます。「assistant」ロールは、ユーザー入力に対するモデルの応答を表します。メッセージ履歴に{"role": "assistant", "content": assistant_msg}を追加するとき、モデルの最後の返答を保存しています。これにより、複数ターンの会話でコンテキストを維持し、モデルが以前に言ったことを記憶して、それに応じて応答できるようになります。

In [ ]:
def example_4_conversation(messages):   
    response = requests.post("http://localhost:11434/api/chat", json={
        "model": "llama3.1:8b",
        "messages": messages,
        "stream": False
    })
    
    if response.status_code == 200:
        assistant_msg = response.json()["message"]["content"]
        return assistant_msg

### 会話を開始：

In [ ]:
print("\n例4: 会話")
print("-" * 30)
messages = [{"role": "system", "content": "あなたは役立つコーディングアシスタントです。"}]
messages.append({"role": "user", "content": "Pythonでファイルを読み込むにはどうすればいいですか？"})
assistant_msg = example_4_conversation(messages)
print(f"アシスタント:\n{assistant_msg}")

### アシスタントを追加：

In [ ]:
messages.append({"role": "assistant", "content": assistant_msg})

### フォローアップの質問を追加：

In [ ]:
messages.append({"role": "user", "content": "エラー処理についてはどうですか？"})
answer = example_4_conversation(messages)
print(f"\n\nフォローアップ:\n{answer}")

✅ すべての例が完了しました！

💡 重要なポイント：

• OpenAIのURLをhttp://localhost:11434に置き換える

• APIキーは不要！

• LangChainでChatOpenAIをChatOllamaに置き換える